<a href="https://colab.research.google.com/github/DreamWalker1412/Colab/blob/main/DCNN_FERPlus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# download cntk & data

In [ ]:
!apt-get install --no-install-recommends openmpi-bin libopenmpi-dev libopencv-dev python3-opencv python-opencv && ln -sf /usr/lib/x86_64-linux-gnu/libmpi_cxx.so /usr/lib/x86_64-linux-gnu/libmpi_cxx.so.1 && ln -sf /usr/lib/x86_64-linux-gnu/openmpi/lib/libmpi.so /usr/lib/x86_64-linux-gnu/openmpi/lib/libmpi.so.12 && ln -sf /usr/lib/x86_64-linux-gnu/libmpi.so /usr/lib/x86_64-linux-gnu/libmpi.so.12 && pip install cntk-gpu

In [ ]:
!git clone https://github.com/microsoft/FERPlus.git

Cloning into 'FERPlus'...
remote: Enumerating objects: 128, done.
remote: Total 128 (delta 0), reused 0 (delta 0), pack-reused 128
Receiving objects: 100% (128/128), 650.37 KiB | 13.84 MiB/s, done.
Resolving deltas: 100% (70/70), done.


In [ ]:
!unzip -n /content/drive/MyDrive/data/fer2013.zip

Archive:  /content/drive/MyDrive/data/fer2013.zip
  inflating: fer2013.csv             


In [ ]:
!mv -n /content/fer2013.csv /content/FERPlus

In [ ]:
!mkdir data
!python /content/FERPlus/src/generate_training_data.py -d /content/FERPlus/data -fer /content/FERPlus/fer2013.csv -ferplus /content/FERPlus/fer2013new.csv

Start generating ferplus images.
Done...


import python file:

```
from google.colab import files
src = list(files.upload().values())[0]
open('mylib.py','wb').write(src)
import mylib
```


```
%load /content/FERPlus/src/models.py
```


```
import sys
sys.path.append('/content/FERPlus/src')


import models
```






# Run the demo code

In [ ]:
!python /content/FERPlus/src/train.py -d /content/FERPlus/data -m probability

# Define the VGG13

In [ ]:
from tensorflow import keras
import tensorflow as tf
from keras.layers import *
from keras.models import Sequential

def get_VGG13():
  model = Sequential(name='VGG13')
  model.add(InputLayer(input_shape=(64,64,1),name='input_1'))
  model.add(Conv2D(64,(3,3),activation='relu',padding='same',name='block1_conv1'))
  model.add(Conv2D(64,(3,3),activation='relu',padding='same',name='block1_conv2'))
  model.add(MaxPooling2D((2,2),name='block1_pool'))
  model.add(Dropout(0.25,name='block1_dropout'))

  model.add(Conv2D(128,(3,3),activation='relu',padding='same',name='block2_conv1'))
  model.add(Conv2D(128,(3,3),activation='relu',padding='same',name='block2_conv2'))
  model.add(MaxPooling2D((2,2),name='block2_pool'))
  model.add(Dropout(0.25,name='block2_dropout'))

  model.add(Conv2D(256,(3,3),activation='relu',padding='same',name='block3_conv1'))
  model.add(Conv2D(256,(3,3),activation='relu',padding='same',name='block3_conv2'))
  model.add(Conv2D(256,(3,3),activation='relu',padding='same',name='block3_conv3'))
  model.add(MaxPooling2D((2,2),name='block3_pool'))
  model.add(Dropout(0.25,name='block3_dropout'))


  model.add(Conv2D(256,(3,3),activation='relu',padding='same',name='block4_conv1'))
  model.add(Conv2D(256,(3,3),activation='relu',padding='same',name='block4_conv2'))
  model.add(Conv2D(256,(3,3),activation='relu',padding='same',name='block4_conv3'))
  model.add(MaxPooling2D((2,2),name='block4_pool'))
  model.add(Dropout(0.25,name='block4_dropout'))

  model.add(Flatten(name='flatten'))
  model.add(Dense(1024,activation='relu',name='fc1'))
  model.add(Dropout(0.5,name='fc_dropout1'))
  model.add(Dense(1024,activation='relu',name='fc2'))
  model.add(Dropout(0.5,name='fc_dropout2'))
  model.add(Dense(8,activation='softmax',name='predictions'))
  return model

# Data preprocessing

In [ ]:
import numpy as np
import ossaudiodev
import matplotlib.pyplot as plt
from PIL import Image
import sys
import csv
import os


def show_img(img_arr):
  img = img_arr.reshape((64,64))
  img = Image.fromarray(img)
  plt.imshow(img ,cmap='gray')
  plt.show()
  return None

def get_image_name_list(dir_path):
  return [os.path.join(dir_path,f) for f in os.listdir(dir_path) if f.endswith('.png')]

def get_image_arr(dir_path, color_mode="grayscale" ,target_size=(64,64)):
  img_list = get_image_name_list(dir_path)
  img_list.sort()
  image_list = []
  for fname in img_list:
    image = keras.preprocessing.image.load_img(fname,color_mode=color_mode,target_size=target_size)
    image_arr = keras.preprocessing.image.img_to_array(image)
    image_list.append(image_arr)
  input_arr = np.array(image_list)
  return input_arr

def get_label(dir_path, mode='crossentropy'):
  fname = 'label.csv'
  label_path = os.path.join(dir_path,fname)
  with open(label_path) as csvfile:
    label_file = csv.reader(csvfile) 
    label_list = []
    for row in label_file:
      emotion_raw = list(map(float, row[2:len(row)]))
      emotion = process_data(emotion_raw=emotion_raw,mode=mode)
      label_list.append(emotion)
  return label_list

def process_data(emotion_raw, mode='crossentropy'):
    '''
    Based on https://arxiv.org/abs/1608.01041, we process the data differently depend on the training mode:

    Majority: return the emotion that has the majority vote, or unknown if the count is too little.
    Probability or Crossentropty: convert the count into probability distribution.abs
    Multi-target: treat all emotion with 30% or more votes as equal.
    '''        
    size = len(emotion_raw)
    emotion_unknown     = [0.0] * size
    emotion_unknown[-2] = 1.0

    # remove emotions with a single vote (outlier removal) 
    for i in range(size):
        if emotion_raw[i] < 1.0 + sys.float_info.epsilon:
            emotion_raw[i] = 0.0

    sum_list = sum(emotion_raw)
    emotion = [0.0] * size 

    if mode == 'majority': 
        # find the peak value of the emo_raw list 
        maxval = max(emotion_raw) 
        if maxval > 0.5*sum_list: 
            emotion[np.argmax(emotion_raw)] = maxval 
        else: 
            emotion = emotion_unknown   # force setting as unknown 
    elif (mode == 'probability') or (mode == 'crossentropy'):
        sum_part = 0
        count = 0
        valid_emotion = True
        while sum_part < 0.75*sum_list and count < 3 and valid_emotion:
            maxval = max(emotion_raw) 
            for i in range(size): 
                if emotion_raw[i] == maxval: 
                    emotion[i] = maxval
                    emotion_raw[i] = 0
                    sum_part += emotion[i]
                    count += 1
                    if i >= 8:  # unknown or non-face share same number of max votes 
                        valid_emotion = False
                        if sum(emotion) > maxval:   # there have been other emotions ahead of unknown or non-face
                            emotion[i] = 0
                            count -= 1
                        break
        if sum(emotion) <= 0.5*sum_list or count > 3: # less than 50% of the votes are integrated, or there are too many emotions, we'd better discard this example
            emotion = emotion_unknown   # force setting as unknown 
    elif mode == 'multi_target':
        threshold = 0.3
        for i in range(size): 
            if emotion_raw[i] >= threshold*sum_list: 
                emotion[i] = emotion_raw[i] 
        if sum(emotion) <= 0.5 * sum_list: # less than 50% of the votes are integrated, we discard this example 
            emotion = emotion_unknown   # set as unknown 
                            
    return [float(i)/sum(emotion) for i in emotion]

def get_data(dir_path,mode='crossentropy'):  # not unknown or non-face 
  imgs = get_image_arr(dir_path)
  labels = get_label(dir_path,mode)
  if len(imgs)==len(labels):
    data_X = []
    data_y = []
    for i in range(len(labels)): 
      emotion = labels[i]
      idx = np.argmax(labels[i])
      if idx < 8: 
          emotion = emotion[:-2]
          emotion = [float(j)/sum(emotion) for j in emotion]
          data_X.append(imgs[i])
          data_y.append(emotion)
    data_X = np.array(data_X)
    data_y = np.array(data_y)
    return data_X,data_y
  else:
    print("Inconsistent number of images and labels")
    return None,None

In [ ]:
import os 

base_dir = '/content/FERPlus/data'
train_dir = os.path.join(base_dir,'FER2013Train')
validation_dir = os.path.join(base_dir,'FER2013Valid')
test_dir = os.path.join(base_dir,'FER2013Test')

In [ ]:
train_X,train_y = get_data(train_dir)
validation_X,validation_y = get_data(validation_dir)
test_X,test_y = get_data(test_dir)

# Fit the model

In [ ]:
model = get_VGG13()
model.compile(optimizer=keras.optimizers.RMSprop(),loss= 'kullback_leibler_divergence', metrics=['kullback_leibler_divergence'])
history = model.fit(train_X, train_y, batch_size=256, epochs=10, validation_data=(validation_X,validation_y))

In [ ]:
import matplotlib.pyplot as plt

loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1,len(loss)+1)

plt.plot(epochs,loss,'ro',label='Training loss')
plt.plot(epochs,val_loss,'r',label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

NameError: ignored

In [ ]:
# create needed folders.
import os,shutil
output_model_path   = os.path.join(base_folder, R'models')
output_model_folder = os.path.join(output_model_path, model_name + '_' + training_mode)
if not os.path.exists(output_model_folder):
    os.makedirs(output_model_folder)